In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint
from bellek.musique.eval import calculate_metrics, compare_answers

set_seed(89)

/home/pc/anaconda3/envs/baris/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

In [9]:
pd.options.display.float_format = '{:,.3f}'.format

In [12]:
from bellek.musique.constants import ABLATION_RECORD_IDS

def report_scores(pred_path: Path):
    df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)
    df.set_index('id', inplace=True, drop=False)
    df = df.loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)
    baseline_df = pd.read_json(pred_path, orient='records', lines=True)
    df = pd.merge(df, baseline_df.drop(columns=['predicted_support_idxs', 'predicted_answerable']), on='id', suffixes=('', ''))
    df = compare_answers(df)
    scores = calculate_metrics(df)
    scores['algo'] = pred_path.stem.replace('musique_ans_v1.0_dev_', '').replace('_model_predictions', '')
    return scores

In [14]:
BASELINE_PREDS_DIR = Path('/home/pc/Documents/baris/bellek/tmp/predictions')

report_df = pd.DataFrame.from_records([report_scores(pred_path) for pred_path in BASELINE_PREDS_DIR.glob('musique_ans_v1.0_*.jsonl')])
report_df.sort_values('exact_match')

,exact_match,f1,algo
1,0.290,0.375,end2end
3,0.430,0.515,select_answer
2,0.530,0.597,step_execution_by_end2end
0,0.620,0.663,step_execution_by_select_answer
